In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import euclidean_distances, manhattan_distances, cosine_distances
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
import kmedoids
from yellowbrick.cluster import KElbowVisualizer

### 1.  

### 2. 

### 3. 

### 4. 

### 5. 

### 6. 

### 7. 

### 8. 

### 9. 

### 10. 